CHỦ ĐỀ NGHIÊN CỨU: MẠNG HỌC SÂU XỬ LÝ DATASET CÂY LÚA NƯỚC
SỬ DỤNG CÁC MẠNG HỌC SÂU, HỌC TĂNG CƯỜNG ĐỂ CẢI THIỆN ẢNH VÀ PHÁT HIỆN BỆNH CÂY LÚA NƯỚC


In [ ]:
# 1. IMPORT THƯ VIỆN CẦN THIẾT
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import sys

In [ ]:
#1.1 TẠO OUTPUT & LOG 
output_dir = 'output'
os.makedirs(output_dir,exist_ok=True)

log_file = open(os.path.join(output_dir,'log_output.txt'),'w')
sys.stdout = log_file

In [ ]:
# 2. CHUẨN BỊ DỮ LIỆU: CHIA DATASET THÀNH TRAIN, VAL, TEST VÀ THỐNG KÊ
original_data_dir = r'F:\tailieu\HỌC_MÁY\datasetriceleaf'  # Thư mục gốc chứa ảnh theo lớp
base_dir = 'rice_data_split'  # Thư mục mới để chứa dữ liệu chia sẵn
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

class_counts = {}  # Lưu số lượng ảnh theo lớp trước chia tách

# Liệt kê tên các lớp và đếm số lượng ảnh
class_names = []
for cls in os.listdir(original_data_dir):
    cls_path = os.path.join(original_data_dir, cls)
    if os.path.isdir(cls_path):
        imgs = os.listdir(cls_path)
        class_counts[cls] = len(imgs)
        class_names.append(cls)
print("Cac lop trong du lieu:")
for cls in sorted(class_names):
    print(f"- {cls}")

#Xóa thư mục chia cũ
if os.path.exists(base_dir):
    print("Dang xoa thu muc cu de chia lai...")
    shutil.rmtree(base_dir)

# Tạo lại thư mục chia dữ liệu
os.makedirs(train_dir)
os.makedirs(val_dir)
os.makedirs(test_dir)


#CHIA DỮ LIỆU
for cls in class_counts.keys():
    imgs = os.listdir(os.path.join(original_data_dir, cls))
    random.shuffle(imgs)
    n_total = len(imgs)
    n_train = int(n_total * 0.7)
    n_val = int(n_total * 0.2)

    for i, img in enumerate(imgs):
        src = os.path.join(original_data_dir, cls, img)
        if i < n_train:
            dst = os.path.join(train_dir, cls)
        elif i < n_train + n_val:
            dst = os.path.join(val_dir, cls)
        else:
            dst = os.path.join(test_dir, cls)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)

#THỐNG KÊ SỐ LƯỢNG ẢNH SAU KHI CHIA
for split in ['train', 'val', 'test']:
    print(f"\nThong ke so anh trong tap {split}:")
    split_path = os.path.join(base_dir, split)
    if os.path.exists(split_path):
        for cls in os.listdir(split_path):
            cls_path = os.path.join(split_path, cls)
            if os.path.isdir(cls_path):
                n = len(os.listdir(cls_path))
                print(f"- {cls}: {n} anh")
    else:
        print(f"Thu muc {split_path} khong ton tai.")

#KIỂM TRA THƯ MỤC ---
print("\nKiem tra ton tai thu muc:")
print("bas dir ton tai:", os.path.exists(base_dir))
print("Train dir ton tai:", os.path.exists(train_dir))
print("Val dir ton tai:", os.path.exists(val_dir))
print("Test dir ton tai:", os.path.exists(test_dir))

# VẼ BIỂU ĐỒ PHÂN BỐ SỐ LƯỢNG ẢNH BAN ĐẦU THEO LỚP
plt.figure(figsize=(10, 5))
sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
plt.title('So luong anh theo lop (truoc khi chia)')
plt.xlabel('Lop benh')
plt.ylabel('So luong anh')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_dir,'initial_class_distribution.png'))
plt.show()


In [ ]:
# 3. TIỀN XỬ LÝ VÀ TĂNG CƯỜNG DỮ LIỆU
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# 4. XÂY DỰNG VÀ FINE-TUNE MÔ HÌNH CNN
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True), #ham hoi tu ngan overfitting
    ReduceLROnPlateau(patience=3, factor=0.2, verbose=1)
]
# 5. HUẤN LUYỆN MÔ HÌNH
history = model.fit(train_generator, validation_data=val_generator, epochs=100, callbacks=callbacks)

In [ ]:
# 6. ĐÁNH GIÁ MÔ HÌNH TRÊN TEST SET
test_generator.reset()
pred = model.predict(test_generator)
y_pred = np.argmax(pred, axis=1)
y_true = test_generator.classes

report = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())
print("Classification Report:")
print(report)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Du doan')
plt.ylabel('Thuc te')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.savefig(os.path.join(output_dir,'confusion_matrix.png'))
plt.show()

In [ ]:
# 7. BIỂU ĐỒ ACCURACY VÀ LOSS
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title("Accuracy theo tung epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.savefig(os.path.join(output_dir,'accuracy_plot.png'))
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss theo tung epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'loss_plot.png'))
plt.show()

In [ ]:
#8. ĐÓNG FILE LOG VÀ KHÔI PHỤC STDOUT
sys.stdout = sys.__stdout__
log_file.close()

In [ ]:
#9. ZIP FOLDER OUTPUT
shutil.make_archive("output",'zip',output_dir)